In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import ourfunctions

from matplotlib import pyplot as plt
import seaborn as sns
#from sklearnex import patch_sklearn
#patch_sklearn(verbose=False)
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import plot_confusion_matrix, recall_score, accuracy_score, precision_score, f1_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier




In [ ]:
X = pd.read_csv('data/Training-set-values.csv')
y = pd.read_csv('data/Training-set-labels.csv')

X['date_recorded'] = pd.to_datetime(X['date_recorded']).astype(np.int64)

#### Preprocessors

In [ ]:
# Super basic numeric transformer

numeric_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='median'))]
)

numeric_preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, make_column_selector(dtype_include=np.number)),
    ]
)

### Models

In [ ]:
# kNearestNeighbors
kNearestNeighbors = {'classifier': KNeighborsClassifier(n_jobs=3), 'preprocessor': None}

# Logistic Regressoion
LogisticRegression = {'classifier': LogisticRegression(C=1e6, n_jobs=3), 'preprocessor': None}

# Decision Trees
DecisionTrees = {'classifier': DecisionTreeClassifier(),'preprocessor': None}
# Decision Trees - adjusted
DecisionTressAdjusted = {'classifier': DecisionTreeClassifier(criterion=['gini','entropy'], max_depth=[90,100], min_samples_split=[2,3], class_weight=['balaced']),'preprocessor': numeric_preprocessor}

# Random Forest with numeric processor
RandomFM_1 = {'classifier': RandomForestClassifier(max_depth=20, min_samples_split=4, n_jobs=3), 'preprocessor': numeric_preprocessor}
# Random Forest no processor
RandomFM_2 = {'classifier': RandomForestClassifier(max_depth=20, min_samples_split=4, n_jobs=3), 'preprocessor': None}
# Random Forest default
# Included for RandomCVSearch later on
RandomFM_rs = {'classifier': RandomForestClassifier(n_jobs=3), 'preprocessor': None}

models = {'kNearestNeighbors': kNearestNeighbors,
    'LogisticRegression': LogisticRegression,
    'DecisionTrees': DecisionTrees,
    'DecisionTressAdjusted': DecisionTressAdjusted,
    'RandomFM_1': RandomFM_1,
    'RandomFM_2': RandomFM_2,
    'RandomFM_rs': RandomFM_rs,
    }


### Modeler

In [ ]:
model_run = ourfunctions.Modeler(models, X=X, y=y)

# after the model_run object is created so we can add onto the default preprocessor.
log_reg_regularized = {'classifier': LogisticRegression(n_jobs=3), 'preprocessor': model_run.create_default_prep(num_add=[('scaling', StandardScaler())])}
model_run.add_model('log_reg_regularized', log_reg_regularized)

### Search parameters and kwargs

In [ ]:
kNN_params = dict(leaf_size=[1,50],
                    n_neighbors=[1,30], 
                    p=[1,2])
                    
LogRegRCV_params = dict(penalty=['l1', 'l2', 'elasticnet'],
                        C=stats.uniform(loc=1, scale=10),
                        max_iter=list(range(100,400)))

DecisionTree_params = dict(criterion=['gini', 'entropy'],
                        max_depth = list(range(20,50)),
                        min_samples_split = list(range(2, 10)))

RandForestRCV_params = dict(n_estimators=list(range(100,300)),
                            criterion=['gini', 'entropy'],
                            max_depth = list(range(20,50)),
                            min_samples_split = list(range(2, 10)))

search_options = {'n_jobs': 3, 'random_state': 9280210, 'n_iter': 20}

## Training LogisticRegression Model

In [ ]:
model_run.train_model('LogisticRegression')

## RandomizedSearchCV

In [ ]:
model_run.hyper_search('kNearestNeighbors', params=kNN_params, searcher_kwargs=search_options, set_to_train=True)

In [ ]:
model_run.hyper_search('log_reg_regularized', params=LogRegRCV_params, searcher_kwargs=search_options, set_to_train=True)

In [ ]:
model_run.hyper_search('DecisionTrees', params=DecisionTree_params, searcher_kwargs=search_options, set_to_train=True)

In [ ]:
model_run.hyper_search('DecisionTressAdjusted', params=DecisionTree_params, searcher_kwargs=search_options, set_to_train=True)

In [ ]:
model_run.hyper_search('RandomFM_1', params=RandForestRCV_params, searcher_kwargs=search_options, set_to_train=True)

In [ ]:
model_run.hyper_search('RandomFM_2', params=RandForestRCV_params, searcher_kwargs=search_options, set_to_train=True)

In [ ]:
model_run.hyper_search('RandomFM_rs', params=RandForestRCV_params, searcher_kwargs=search_options, set_to_train=True)

## Test Models

In [ ]:
model_run.test_all()

## Plotting

In [ ]:
model_run.plot_models(save='baseline_models_graph')

## Modeler

### Random Forests

In [ ]:
model_run.model_evaluation('')

In [ ]:
importance_kwargs = dict(n_repeats=10, n_jobs=3)
model_run.permutation_importance('RandomFM_default', perm_kwargs=importance_kwargs)

### Logistic Regression

In [ ]:
model_run.model_evaluation('')

In [ ]:
importance_kwargs = dict(n_repeats=10, n_jobs=3)
model_run.permutation_importance('log_reg_regularized', perm_kwargs=importance_kwargs)